# Определение токсичных комментариев для английских текстов

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Нужно обучить модель классифицировать комментарии на позитивные и негативные. Для этой задачи создан набор данных с разметкой о токсичности правок.

Нужно построить модель со значением метрики качества *F1* не меньше 0.75. 

**Шаги**

1. Загрузка и подготовка данных.
2. Обучение разных моделей. 
3. Выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. 

Столбец *text* — текст комментария, а *toxic* — целевой признак.

## Импорт библиотек

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
import nltk
import re

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline

## Загрузка, подготовка и анализ данных

Загрузка данных и их предварительный просмотр:

In [3]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
except:
    data = pd.read_csv('D:\\data\\toxic_comments.csv')

In [4]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Есть странная колонка "Unnamed: 0". С первого взгляда кажется, что она дублирует индексы. Проверю, так ли это:

In [5]:
data.index.equals(data['Unnamed: 0'])

False

Нет, дублирования нет. Проверю, где значения начинают отличаться друг от друга (номер индекса):

In [6]:
for index, (idx, value) in enumerate(data['Unnamed: 0'].items()):
    if idx != value:
        first_mismatch_index = idx
        break
        
first_mismatch_index

6080

In [7]:
data.loc[6078:6082]

,Unnamed: 0,text,toxic
6078,6078,"""\n\nPenis envy\nFrom Wikipedia, the free ency...",1
6079,6079,"Hrafn \n\nPM, Where did Hrafn go? There was a ...",0
6080,6084,"""::I'll alos be looking in to see how this is ...",0
6081,6085,"""\n\nThe Ezekiel passage is quoted in the Molo...",0
6082,6086,Thank you for experimenting with Wikipedia. Y...,0


Здесь видно, что на индексе 6080 значение колонки unnamed уже не равно номеру индекса. Эту колонку можно удалить, так как она никак не поможет обучению модели. Это было и так понятно, но я решила проверить, почему колонка похожа на индексы, но отличается от них:

In [8]:
data.drop(columns=data.columns[0], inplace=True)

Информация о количестве данных, пропусках и типах данных:

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [10]:
print("Количество дубликатов:", data.duplicated().sum())

Количество дубликатов: 0


**Промежуточный вывод:** удалена колонка Unnamed: 0, данные проверены на дубликаты и пропуски - их нет. Также просмотрены типы данных, все в порядке. Можно приступать к подготовке к обучению.

## Подготовка к обучению

Баланс классов:

In [19]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Есть явный дисбаланс классов, я применю технику оверсемплинга для тренировочного набора данных.

Создам корпус слов:

In [20]:
corpus = data['text']

Сделаю лемматизацию английского текста с помощью WordNetLemmatizer и POS-тегов:

In [21]:
%%time

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Если тег не распознается

def lemmatize_text(text):
    tagged_words = nltk.pos_tag(nltk.word_tokenize(text))
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(pos_tag)) for word, pos_tag in tagged_words]
    return " ".join(lemmatized_words)

corpus = pd.Series(corpus)

lemmatized_corpus = corpus.apply(lemmatize_text)

print("Исходный текст:", corpus[2])
print("Лематизированный текст:", lemmatized_corpus[2])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Катя\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Катя\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Катя\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Исходный текст: Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.
Лематизированный текст: Hey man , I 'm really not try to edit war . It 's just that this guy be constantly remove relevant information and talk to me through edits instead of my talk page . He seem to care more about the formatting than the actual info .
Wall time: 12min 56s


Уберу лишние символы и приведу массив к нижнему регистру:

In [22]:
%%time

def clear_text(text):
    edited = re.sub(r'[^a-zA-Z ]', ' ', text).split()
    return " ".join(edited)

def lowercase_text(text):
    lowercase_words = [word.lower() for word in text.split()]
    return " ".join(lowercase_words)

cleared_corpus = lemmatized_corpus.apply(clear_text)
lowercased_corpus = cleared_corpus.apply(lowercase_text)

print("Исходный текст:", corpus[2])
print("Очищенный, лемматизированный текст:", lowercased_corpus[2])

Исходный текст: Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.
Очищенный, лемматизированный текст: hey man i m really not try to edit war it s just that this guy be constantly remove relevant information and talk to me through edits instead of my talk page he seem to care more about the formatting than the actual info
Wall time: 4.62 s


Разделю данные на train и test:

In [23]:
x_train, x_test, y_train, y_test = train_test_split(lowercased_corpus, data['toxic'], test_size=0.2, random_state=42)

# Проверка
print("Размеры выборок:")
print("Train:", x_train.shape[0], y_train.shape[0])
print("Test:", x_test.shape[0], y_test.shape[0])

Размеры выборок:
Train: 127433 127433
Test: 31859 31859


Скачиваю стоп-слова и присваиваю их переменной:

In [25]:
nltk.download('stopwords') #скачивание стоп-слов из библиотеки

english_stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Катя\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Обучение

Создаю пайплайны для 3 моделей и делаю для них сетку гиперпараметров. Буду использовать векторайзер, также tf-idf трансформер, smote балансировщик.

In [26]:
# Создание пайплайна для каждой модели
logreg_pipe = make_pipeline(
    CountVectorizer(ngram_range=(1, 2), stop_words=english_stopwords),
    TfidfTransformer(),
    SMOTE(random_state=42),
    LogisticRegression(random_state=42)
)

rf_pipe = make_pipeline(
    CountVectorizer(ngram_range=(1, 2), stop_words=english_stopwords),
    TfidfTransformer(),
    SMOTE(random_state=42),
    RandomForestClassifier(random_state=42)
)

lgbm_pipe = make_pipeline(
    CountVectorizer(ngram_range=(1, 2), stop_words=english_stopwords),
    TfidfTransformer(),
    SMOTE(random_state=42),
    LGBMClassifier(random_state=42)
)


# Создание сетки параметров для каждой модели
logreg_params = {
    'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'logisticregression__solver': ['liblinear', 'saga'],
    'logisticregression__max_iter': range(100, 1000, 100),
    'logisticregression__fit_intercept': [True, False],
    'logisticregression__penalty': ['l1', 'l2']
}


rf_params = {
    'randomforestclassifier__n_estimators': [100, 200, 300, 400, 500],
    'randomforestclassifier__criterion': ['gini', 'entropy'],
    'randomforestclassifier__max_depth': [None, 5, 10, 20, 30],
    'randomforestclassifier__min_samples_split': [2, 5, 10],
    'randomforestclassifier__min_samples_leaf': [1, 2, 4],
    'randomforestclassifier__max_features': ['auto', 'sqrt', 'log2'],
    'randomforestclassifier__bootstrap': [True, False]
}

lgbm_params = {
    'lgbmclassifier__n_estimators': range(500, 1001, 100),
    'lgbmclassifier__max_depth': range(5, 16),
    'lgbmclassifier__learning_rate': [0.01, 0.1, 0.5, 1.0]
}

Считаю метрику для LogisticRegression:

In [27]:
logreg_grid = RandomizedSearchCV(logreg_pipe, logreg_params, cv=3, scoring='f1', verbose=10, n_jobs=-1, random_state=42)

logreg_grid.fit(x_train, y_train)
print("Лучшие гиперпараметры LogisticRegression: ", logreg_grid.best_params_)
print("Лучшая F1-мера: ", logreg_grid.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Лучшие гиперпараметры LogisticRegression:  {'logisticregression__solver': 'liblinear', 'logisticregression__penalty': 'l2', 'logisticregression__max_iter': 500, 'logisticregression__fit_intercept': False, 'logisticregression__C': 100}
Лучшая F1-мера:  0.7608801004373702


Считаю метрику для RandomForestClassifier:

In [28]:
rf_grid = RandomizedSearchCV(rf_pipe, rf_params, cv=3, scoring='f1', verbose=10, n_jobs=-1, random_state=42)

rf_grid.fit(x_train, y_train)
print("Лучшие гиперпараметры RandomForestClassifier: ", rf_grid.best_params_)
print("Лучшая F1-мера: ", rf_grid.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Лучшие гиперпараметры RandomForestClassifier:  {'randomforestclassifier__n_estimators': 400, 'randomforestclassifier__min_samples_split': 5, 'randomforestclassifier__min_samples_leaf': 2, 'randomforestclassifier__max_features': 'auto', 'randomforestclassifier__max_depth': 30, 'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__bootstrap': True}
Лучшая F1-мера:  0.38499067561673206


Считаю метрику для LGBMClassifier:

In [29]:
lgbm_grid = RandomizedSearchCV(lgbm_pipe, lgbm_params, cv=3, scoring='f1', verbose=10, n_jobs=-1, random_state=42)

lgbm_grid.fit(x_train, y_train)
print("Лучшие гиперпараметры LGBMClassifier: ", lgbm_grid.best_params_)
print("Лучшая F1-мера: ", lgbm_grid.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Лучшие гиперпараметры LGBMClassifier:  {'lgbmclassifier__n_estimators': 900, 'lgbmclassifier__max_depth': 12, 'lgbmclassifier__learning_rate': 0.5}
Лучшая F1-мера:  0.7529564425229078


**Лучшая модель** - логистическая регрессия с параметрами 'solver': 'liblinear', 'penalty': 'l2', 'max_iter': 500, 'fit_intercept': False, 'C': 100.

## Тестирование лучшей модели

LogisticRegression:

In [30]:
y_pred = logreg_grid.best_estimator_.predict(x_test)
print("F1 на тесте:", f1_score(y_test, y_pred))

F1 на тесте: 0.7549497847919656


Проверка на адекватность с предсказанием классов в соответствии с их относительными частотами в тренировочных данных:

Для данных не нужно применять трансформер или векторайзер, потому что по сути предсказание никак не зависит от признаков. Оно строится только на таргете со случайным предсказанием классов.

In [31]:
model_dm = DummyClassifier(strategy='stratified', random_state=42)

model_dm.fit(x_train, y_train)

predicted_dm = model_dm.predict(x_test)

print("F1 на Dummy:", f1_score(y_test, predicted_dm))

F1 на Dummy: 0.09324009324009325


Результат F1 модели значительно лучше, чем результат Dummy.

## Выводы

1. Данные были загружены и проверены, удалена одна колонка.
2. Данные были изучены и подготовлены для обучения с помощью лемматизации, приведения к нижнему регистру, удалению стоп-слов, а также созданию мешка слов.
3. Рассчитаны гиперпараметры для 3 моделей: логистическая регрессия, случайный лес, lgdm классификатор. Лучше всего себя проявила логистическая регрессия с гиперпараметрами 'solver': 'liblinear', 'penalty': 'l2', 'max_iter': 500, 'fit_intercept': False, 'C': 100.
4. Лучшая модель проверена на тесте. Результат F1 = 0.755
5. Проведена проверка на адекватность Dummy классификатором. F1 = 0.093

**Выбранная модель:** LogisticRegression(random_state=42, solver='liblinear', penalty='l2', max_iter=500, fit_intercept=False, C=100). Тестирование проведено успешно.